In [2]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


ModuleNotFoundError: No module named 'pandas'

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from PIL import Image
import librosa
import librosa.display
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


In [ ]:
# # Directory containing the audio files
# directory = 'Data/genres_original'
# # Directory to save the MEL spectrogram images
# output_directory = 'mel_spectrograms'

# # Create the output directory if it doesn't exist
# os.makedirs(output_directory, exist_ok=True)

# # Iterate over all subdirectories and files in the directory
# for root, dirs, files in os.walk(directory):
#     for file in files:
#         if file.endswith('.wav'):  # Assuming the audio files are in .wav format
#             file_path = os.path.join(root, file)
#             try:
#                 # Load the audio file
#                 y, sr = librosa.load(file_path, mono=True, sr=None)
#                 # Extract MEL spectrogram
#                 mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
#                 # Convert to log scale (dB)
#                 log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
                
#                 # Plot the MEL spectrogram
#                 plt.figure(figsize=(10, 4))
#                 librosa.display.specshow(log_mel_spectrogram, sr=sr, x_axis='time', y_axis='mel')
#                 plt.colorbar(format='%+2.0f dB')
#                 plt.title('MEL Spectrogram')
#                 plt.tight_layout()
                
#                 # Save the plot as a PNG file
#                 output_file_path = os.path.join(output_directory, f"{os.path.splitext(file)[0]}.png")
#                 plt.savefig(output_file_path)
#                 plt.close()
                
#                 print(f"Saved MEL spectrogram for {file_path} as {output_file_path}")
#             except Exception as e:
#                 print(f"Error processing {file_path}: {e}")

In [ ]:
# Directory containing the MEL spectrogram images
directory = 'mel_spectrograms'

# List to store the data
data = []

# Iterate over all files in the directory
for file in os.listdir(directory):
    if file.endswith('.png'):  # Assuming the spectrograms are in .png format
        file_path = os.path.join(directory, file)
        # Extract the genre from the filename (word before the first '.')
        genre = file.split('.')[0]
        # Append the data (filename, genre, file path) to the list
        data.append([file, genre, file_path])

# Create a DataFrame from the data
columns = ['filename', 'genre', 'spectrogram']
df_ms = pd.DataFrame(data, columns=columns)

# Display the first few rows of the DataFrame
print(df_ms.info())

In [ ]:
# Display the first few rows of the DataFrame
print(df_ms.head())

# Display basic statistics
print(df_ms.describe())

# Check for missing values
print(df_ms.isnull().sum())

# Plot the distribution of genres
plt.figure(figsize=(10, 6))
df_ms['genre'].value_counts().plot(kind='bar')
plt.title('Distribution of Genres')
plt.xlabel('Genre')
plt.ylabel('Count')
plt.show()

# Plot the distribution of spectrogram file sizes
file_sizes = df_ms['spectrogram'].apply(lambda x: os.path.getsize(x))
plt.figure(figsize=(10, 6))
plt.hist(file_sizes, bins=30, edgecolor='k')
plt.title('Distribution of Spectrogram File Sizes')
plt.xlabel('File Size (bytes)')
plt.ylabel('Count')
plt.show()

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from sklearn.model_selection import train_test_split
# from tqdm import tqdm
# import warnings
# warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# Initialize lists to store features and labels
features = []
labels = []

# Iterate over the DataFrame rows
for index, row in tqdm(df_ms.iterrows(), total=df_ms.shape[0]):
    # Load the image
    img = Image.open(row['spectrogram'])
    # Convert the image to a numpy array
    img_array = np.array(img)
    # Append the image array to the features list
    features.append(img_array)
    # Append the genre (label) to the labels list
    labels.append(row['genre'])

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Print the shapes of the arrays
print(f"Features shape: {features.shape}")
print(f"Labels shape: {labels.shape}")

In [ ]:
# Normalize the features
features = features / 255.0

# Convert labels to categorical (one-hot encoding)
unique_labels = np.unique(labels)
label_to_index = {label: index for index, label in enumerate(unique_labels)}
labels = np.array([label_to_index[label] for label in labels])
labels = to_categorical(labels, num_classes=len(unique_labels))

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(features.shape[1], features.shape[2], features.shape[3])),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(unique_labels), activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Use ImageDataGenerator to load data in batches and perform data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Use a portion of the data for validation
)

train_generator = datagen.flow_from_dataframe(
    dataframe=df_ms,
    directory='mel_spectrograms',
    x_col='spectrogram',
    y_col='genre',
    target_size=(features.shape[1], features.shape[2]),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=df_ms,
    directory='mel_spectrograms',
    x_col='spectrogram',
    y_col='genre',
    target_size=(features.shape[1], features.shape[2]),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Train the model with detailed feedback
model.fit(train_generator, epochs=10, validation_data=validation_generator, verbose=1)

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Test accuracy: {accuracy}")